# Brief

# TODO

At every boot, copy everything from google drive instead of third party websites. Then, after all work is done, automatically save everything to google drive. Don't forget NOT TO SAVE hundreds of thousands of small files, but archives (tar) instead

In [ ]:
from google.colab import drive
drive.mount('/drive')
!mkdir -p /drive/MyDrive/vehicle_detection_and_classification
!ls -la /drive/MyDrive/

In [ ]:
!apt-get -q install tree

%pip install seaborn

# TODO needed?:
%pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

In [1]:
from pprint import pprint
import numpy as np
import cv2
import csv
import json
import os
import sys
import shutil
import pickle

In [ ]:
drive_path = "/drive/MyDrive/"
drive_traffic_cams_path = os.path.join(drive_path, "traffic_cams")
drive_local_proj_path = os.path.join(drive_path, "tedros_laptop/Desktop/d_projekty/bp")
drive_proj_path = os.path.join(drive_path, "vehicle_detection_and_classification")
drive_proj_datasets_path = os.path.join(drive_proj_path, "datasets")
proj_path = "/content/"
datasets_path = os.path.join(proj_path, "datasets")
models_path = os.path.join(proj_path, "models")

scripts_path = os.path.join(drive_proj_path, "scripts")
working_path = os.path.join(drive_proj_path, "working_dir")

!ls $drive_traffic_cams_path
!echo =====
!ls $drive_local_proj_path

!mkdir -p $drive_proj_path
!mkdir -p $proj_path
!mkdir -p $datasets_path
!mkdir -p $models_path
!mkdir -p $working_path

# Define, download and process datasets

In [ ]:
# TODO join car and trailer to car_and_trailer
classes = [
    "pedestrian", 
    "bicycle", 
    "motorcycle", 
    "passenger_car",
    "transporter",
    "bus",
    "truck",
    "unknown"
]
classes_dict = {
    0: "pedestrian",
    1: "bicycle", 
    2: "motorcycle", 
    3: "passenger_car",
    4: "transporter",
    5: "bus",
    6: "truck",
    7: "unknown"
}



## MIO-TCD

- [https://tcd.miovision.com/challenge/dataset.html](https://tcd.miovision.com/challenge/dataset.html)
- ~650k objects on ~130k frames (test data are not annotated, so just 110k usable frames)
- Classes: Pedestrian + 9 vehicle classes + background

In [ ]:
dataset_mio_tcd_name = "MIO-TCD-Localization"
dataset_mio_tcd_path = os.path.join(datasets_path, dataset_mio_tcd_name)

# Download directly from the server
# !wget -c https://tcd.miovision.com/static/dataset/MIO-TCD-Localization.tar -P $datasets_path

# Download to your google drive and then only download from the google drive
# !wget -c https://tcd.miovision.com/static/dataset/MIO-TCD-Localization.tar -P $drive_proj_datasets_path
src = os.path.join(drive_proj_datasets_path, dataset_mio_tcd_name + ".tar")
dst = os.path.join(datasets_path, dataset_mio_tcd_name + ".tar")
shutil.copy(src, dst)

In [ ]:
# This takes hours on google drive due to their limits! So we do it on colab disk
!tar -xf $datasets_path/MIO-TCD-Localization.tar -C $datasets_path

!ls -la $dataset_mio_tcd_path

In [ ]:
# This can take several minutes
mio_tcd_process_script_path = os.path.join(scripts_path, "process_dataset", "mio_tcd.py")
!python3 $mio_tcd_process_script_path

### Combine datasets

In [ ]:
combine_datasets_script_path = os.path.join(scripts_path, "process_dataset", "combine_datasets.py")

# Installing mmdetection

In [ ]:
%cd $models_path
!pip install openmim
!mim install mmcv-full
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .
models_mmdetection_path = os.path.join(models_path, "mmdetection")

/content/models
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
fatal: destination path 'mmdetection' already exists and is not an empty directory.
/content/models/mmdetection
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/models/mmdetection
  Attempting uninstall: mmdet
    Found existing installation: mmdet 2.25.3
    Can't uninstall 'mmdet'. No files were found to uninstall.
  Running setup.py develop for mmdet


In [ ]:
#@title Verify installation
import mmdet
print(mmdet.__version__)

2.25.3


/usr/local/lib/python3.7/dist-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [ ]:
from mmcv import collect_env
collect_env()

{'sys.platform': 'linux',
 'Python': '3.7.15 (default, Oct 12 2022, 19:14:55) [GCC 7.5.0]',
 'CUDA available': True,
 'GPU 0': 'Tesla T4',
 'CUDA_HOME': '/usr/local/cuda',
 'NVCC': 'Cuda compilation tools, release 11.2, V11.2.152',
 'GCC': 'x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0',
 'PyTorch': '1.9.0+cu111',
 'PyTorch compiling details': 'PyTorch built with:\n  - GCC 7.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.1.2 (Git Hash 98be7e8afa711dc9b66c8ff3504129cb82013cdb)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.1\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86

# Importing YOLOX-s

In [ ]:
#@title Download YOLOX-x config and checkpoint (pre-trained)
# Memory requirements for YOLOX models: 
# https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_s_8x8_300e_coco/
# YOLOX-s	7.6 GB
# YOLOX-l	19.9 GB
# YOLOX-x	28.1 GB
# While Google Colab provides us with just ~13 GB!

mmdetection_config_path = os.path.join(models_mmdetection_path, "configs")
mmdetection_checkpoint_path = os.path.join(models_mmdetection_path, "checkpoints")

# yolox_config_filename = "yolox_x_8x8_300e_coco.py"
yolox_config_filename = "yolox_s_8x8_300e_coco.py"
yolox_config_filepath = os.path.join(mmdetection_config_path, "yolox", yolox_config_filename)

# And YOLOX-x checkpoint
# yolox_checkpoint_filename = "yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth"
yolox_checkpoint_filename = "yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth"
yolox_checkpoint_filepath = os.path.join(models_mmdetection_path, "checkpoints", yolox_checkpoint_filename)
if not os.path.exists(mmdetection_checkpoint_path):
    os.mkdir(mmdetection_checkpoint_path)
# url = "https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/" + yolox_checkpoint_filename
url = "https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_s_8x8_300e_coco/" + yolox_checkpoint_filename
!wget -c $url -O $yolox_checkpoint_filepath

--2022-11-15 16:58:36--  https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_l_8x8_300e_coco/yolox_l_8x8_300e_coco_20211126_140236-d3bd2b23.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.43
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217289556 (207M) [application/octet-stream]
Saving to: ‘/content/models/mmdetection/checkpoints/yolox_l_8x8_300e_coco_20211126_140236-d3bd2b23.pth’

            /conten  79%[==============>     ] 165.76M  8.24MB/s    eta 6s     

In [ ]:
# https://colab.research.google.com/github/ZwwWayne/mmdetection/blob/update-colab/demo/MMDet_Tutorial.ipynb#scrollTo=hamZrlnH-YDD
from mmdet.apis import set_random_seed
from mmcv import Config

classes_tuple = tuple(classes)

cfg = Config.fromfile(yolox_config_filepath)

# print(f'Initial config:\n{cfg.pretty_text}')

cfg.load_from = yolox_checkpoint_filepath

cfg.work_dir = working_path
cfg.data_root = dataset_mio_tcd_path
# cfg.data_root = datasets_path

cfg.dataset_type = "CustomDataset"

img_prefix = dataset_mio_tcd_path
# img_prefix = datasets_path
ann_file = os.path.join(dataset_mio_tcd_path, "gt.pickle")
# ann_file = os.path.join(datasets_path, "dataset.pickle")
cfg.train_dataset.dataset.type = "CustomDataset"
cfg.train_dataset.dataset.ann_file = ann_file 
cfg.train_dataset.dataset.img_prefix = img_prefix
cfg.data.train.dataset.type = "CustomDataset"
cfg.data.train.dataset.ann_file = ann_file
cfg.data.train.dataset.img_prefix = img_prefix
cfg.data.val.type = "CustomDataset"
cfg.data.val.ann_file = ann_file
cfg.data.val.img_prefix = img_prefix
cfg.data.test.type = "CustomDataset"
cfg.data.test.ann_file = ann_file
cfg.data.test.img_prefix = img_prefix

cfg.gpu_ids = [0]
cfg.device = "cuda"
cfg.device = "cpu"

cfg.seed = 0
set_random_seed(0, deterministic=False)

        
# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'

# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1

cfg.max_epochs = 10
runner = dict(type='EpochBasedRunner', max_epochs=cfg.max_epochs)

# We can also use tensorboard to log the training process
# cfg.log_config.hooks = [
#     dict(type='TextLoggerHook'),
#     dict(type='TensorboardLoggerHook')]

# cfg.train_dataset = {}
# cfg.pop("train_dataset")
# cfg.train_dataset.filter_empty_gt = True

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

In [ ]:
# https://colab.research.google.com/github/ZwwWayne/mmdetection/blob/update-colab/demo/MMDet_Tutorial.ipynb#scrollTo=hamZrlnH-YDD
import mmcv
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
train_detector(model, datasets, cfg)

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
!nvidia-smi

!pip install numba

from numba import cuda 
device = cuda.get_current_device()
device.reset()

# Keepalive

In [ ]:
from time import sleep
counter = 0
while True:
    sleep(10)
    print(counter, end="\r")
    counter += 10